

---

### **1. Что такое N-граммная модель?**
N-граммная модель — это статистическая модель языка, которая предсказывает вероятность появления слова на основе предыдущих \(N-1\) слов. Например:
- Для биграммной модели (\(N=2\)) вероятность слова зависит от одного предыдущего слова.
- Для триграммной модели (\(N=3\)) вероятность слова зависит от двух предыдущих слов.

Цель модели — оценить вероятность последовательности слов и сгенерировать текст, который выглядит как естественный язык.

---

### **2. Основные этапы решения задачи**

#### **2.1. Подготовка данных**
Перед тем как строить модель, текст нужно обработать:
- **Добавление специальных токенов**: В начале и конце каждого предложения добавляются специальные токены `<s>` (начало предложения) и `</s>` (конец предложения). Это помогает модели понимать, где начинаются и заканчиваются предложения.
- **Замена редких слов**: Слова, которые встречаются только один раз в тексте, заменяются на `<UNK>` (неизвестное слово). Это помогает модели справляться с редкими словами и уменьшает размер словаря.

В коде это реализовано функциями:
- `add_sentence_tokens`: добавляет `<s>` и `</s>` к предложениям.
- `replace_singletons`: заменяет редкие слова на `<UNK>`.
- `preprocess`: объединяет эти шаги.

#### **2.2. Построение N-грамм**
После обработки текста создаются N-граммы — последовательности из \(N\) слов. Например, для триграмм:
- Исходный текст: "I love programming."
- Триграммы: `('<s>', '<s>', 'I')`, `('<s>', 'I', 'love')`, `('I', 'love', 'programming')`, `('love', 'programming', '</s>')`.

В коде это делается с помощью библиотеки `nltk`:
- `nltk.ngrams` создает N-граммы.
- `nltk.FreqDist` подсчитывает частоты N-грамм.

#### **2.3. Сглаживание**
Сглаживание — это метод, который помогает избежать проблемы нулевых вероятностей. Если в обучающем тексте не встречается какая-то N-грамма, её вероятность будет равна нулю, что плохо для модели. В этом коде используется **аддитивное сглаживание** (с параметром \(k=1\)):
\[
P(w_n | w_{n-1}, ..., w_1) = \frac{\text{count}(w_1, ..., w_n) + k}{\text{count}(w_1, ..., w_{n-1}) + k \cdot |V|}
\]
где:
- \(\text{count}(w_1, ..., w_n)\) — частота N-граммы.
- \(\text{count}(w_1, ..., w_{n-1})\) — частота (N-1)-граммы.
- \(|V|\) — размер словаря (количество уникальных слов).
- \(k\) — параметр сглаживания (в данном случае \(k=1\)).

Сглаживание реализовано в цикле, где вычисляются вероятности для каждой N-граммы.

#### **2.4. Генерация текста**
Генерация текста происходит следующим образом:
1. Начинаем с токена `<s>` (или нескольких `<s>` для \(N > 1\)).
2. На каждом шаге выбираем следующее слово, основываясь на вероятностях, рассчитанных для текущего контекста (предыдущих \(N-1\) слов).
3. Повторяем, пока не достигнем токена `</s>` (конец предложения) или максимальной длины предложения.

Функция `_best_candidate` выбирает наиболее вероятное следующее слово, исключая запрещенные слова (например, `<UNK>` или `</s>`, если предложение еще слишком короткое).

---

### **3. Концептуальное объяснение кода**

#### **3.1. Импорт библиотек**
- `nltk` используется для работы с текстом (токенизация, подсчет частот, создание N-грамм).
- `spacy` используется для разбиения текста на предложения.
- `math` используется для вычисления логарифмов (при расчете вероятностей).

#### **3.2. Загрузка и обработка текста**
Текст загружается из файла `ml_text.txt`. Затем он разбивается на предложения с помощью `spacy`, и каждое предложение обрабатывается:
- Добавляются токены `<s>` и `</s>`.
- Редкие слова заменяются на `<UNK>`.

#### **3.3. Построение N-грамм**
- Создаются N-граммы и (N-1)-граммы.
- Подсчитываются их частоты.

#### **3.4. Расчет вероятностей**
Для каждой N-граммы рассчитывается вероятность с использованием аддитивного сглаживания. Вероятности хранятся в словаре `probabilities`.

#### **3.5. Генерация текста**
Текст генерируется, начиная с токена `<s>`. На каждом шаге выбирается следующее слово, пока не достигнем конца предложения (`</s>`).

---

### **4. Важные аспекты**

1. **N-граммы**: Основная идея модели — предсказывать слово на основе предыдущих \(N-1\) слов.
2. **Сглаживание**: Необходимость сглаживания для обработки редких или отсутствующих N-грамм.
3. **Обработка текста**: Добавление специальных токенов и замена редких слов.
4. **Генерация текста**: Использование вероятностей для выбора следующего слова.

---


In [ ]:
import nltk
from pprint import pprint
import math

import spacy
nlp = spacy.load("en_core_web_sm")

## Resources
## n-gram model: https://github.com/joshualoehr/ngram-language-model/tree/master

In [ ]:
text = open("./data/ml_text.txt").read()
print(text[0:150], "...")

Machine learning is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from ...


In [ ]:
SOS = "<s> "
EOS = "</s>"
UNK = "<UNK>"

def add_sentence_tokens(sentences, n):
    """Wrap each sentence in SOS and EOS tokens.

    For n >= 2, n-1 SOS tokens are added, otherwise only one is added.

    Args:
        sentences (list of str): the sentences to wrap.
        n (int): order of the n-gram model which will use these sentences.
    Returns:
        List of sentences with SOS and EOS tokens wrapped around them.

    """
    sos = SOS * (n-1) if n > 1 else SOS
    return ['{}{} {}'.format(sos, s, EOS) for s in sentences]

def replace_singletons(tokens):
    """Replace tokens which appear only once in the corpus with <UNK>.

    Args:
        tokens (list of str): the tokens comprising the corpus.
    Returns:
        The same list of tokens with each singleton replaced by <UNK>.

    """
    vocab = nltk.FreqDist(tokens)
    return [token if vocab[token] > 1 else UNK for token in tokens]

def preprocess(sentences, n):
    """Add SOS/EOS/UNK tokens to given sentences and tokenize.

    Args:
        sentences (list of str): the sentences to preprocess.
        n (int): order of the n-gram model which will use these sentences.
    Returns:
        The preprocessed sentences, tokenized by words.

    """
    sentences = add_sentence_tokens(sentences, n)
    tokens = ' '.join(sentences).split(' ')
    tokens = replace_singletons(tokens)
    return tokens

In [ ]:
doc = nlp(text)
sentences = [s.text.lower() for s in doc.sents]
sentences

['machine learning is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.',
 'within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.',
 'ml finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.',
 'the application of ml to business problems is known as predictive analytics.',
 'statistics and mathematical optimization methods comprise the foundations of machine learning.',
 'data mining is a related field of study, focusing on exploratory data analysis via unsupervised learning.',
 'from a theoretical viewpoint, probably approximately correct learning provides a 

In [ ]:
processed_tokens = preprocess(sentences, 3)
processed_tokens

['<s>',
 '<s>',
 'machine',
 'learning',
 'is',
 'a',
 'field',
 'of',
 'study',
 'in',
 'artificial',
 'intelligence',
 'concerned',
 'with',
 'the',
 '<UNK>',
 'and',
 'study',
 'of',
 'statistical',
 'algorithms',
 'that',
 'can',
 'learn',
 'from',
 'data',
 'and',
 'generalize',
 'to',
 'unseen',
 'data,',
 'and',
 'thus',
 'perform',
 'tasks',
 'without',
 'explicit',
 '<UNK>',
 '</s>',
 '<s>',
 '<s>',
 'within',
 'a',
 '<UNK>',
 'in',
 'machine',
 'learning,',
 'advances',
 'in',
 'the',
 'field',
 'of',
 'deep',
 'learning',
 'have',
 '<UNK>',
 'neural',
 'networks,',
 'a',
 'class',
 'of',
 'statistical',
 'algorithms,',
 'to',
 'surpass',
 'many',
 'previous',
 'machine',
 'learning',
 'approaches',
 'in',
 '<UNK>',
 '</s>',
 '<s>',
 '<s>',
 'ml',
 'finds',
 'application',
 'in',
 'many',
 'fields,',
 'including',
 'natural',
 'language',
 'processing,',
 'computer',
 'vision,',
 'speech',
 'recognition,',
 '<UNK>',
 'filtering,',
 '<UNK>',
 'and',
 '<UNK>',
 '</s>',
 '<s>',


In [ ]:
vocabulary  = nltk.FreqDist(processed_tokens)
vocabulary

FreqDist({'<UNK>': 1603, '<s>': 716, 'the': 370, '</s>': 358, 'of': 256, 'a': 242, 'to': 221, 'and': 201, 'in': 181, 'learning': 156, ...})

In [ ]:
N = 3
vocab_size = len(vocabulary)
print(vocab_size)

n_grams = nltk.ngrams(processed_tokens, N)
n_vocab = nltk.FreqDist(n_grams)

m_grams = nltk.ngrams(processed_tokens, N-1)
m_vocab = nltk.FreqDist(m_grams)

print(n_vocab)
print(m_vocab)

850
<FreqDist with 6704 samples and 9070 outcomes>
<FreqDist with 4233 samples and 9071 outcomes>


In [ ]:
for n_gram, c in n_vocab.items():
    print(n_gram, c)

('<s>', '<s>', 'machine') 11
('<s>', 'machine', 'learning') 11
('machine', 'learning', 'is') 8
('learning', 'is', 'a') 6
('is', 'a', 'field') 1
('a', 'field', 'of') 2
('field', 'of', 'study') 2
('of', 'study', 'in') 1
('study', 'in', 'artificial') 1
('in', 'artificial', 'intelligence') 1
('artificial', 'intelligence', 'concerned') 1
('intelligence', 'concerned', 'with') 1
('concerned', 'with', 'the') 1
('with', 'the', '<UNK>') 5
('the', '<UNK>', 'and') 5
('<UNK>', 'and', 'study') 1
('and', 'study', 'of') 1
('study', 'of', 'statistical') 1
('of', 'statistical', 'algorithms') 1
('statistical', 'algorithms', 'that') 1
('algorithms', 'that', 'can') 1
('that', 'can', 'learn') 1
('can', 'learn', 'from') 1
('learn', 'from', 'data') 1
('from', 'data', 'and') 2
('data', 'and', 'generalize') 1
('and', 'generalize', 'to') 1
('generalize', 'to', 'unseen') 1
('to', 'unseen', 'data,') 1
('unseen', 'data,', 'and') 1
('data,', 'and', 'thus') 1
('and', 'thus', 'perform') 1
('thus', 'perform', 'tasks') 

In [ ]:
k = 1
probabilities = {}
for n_gram, n_count in n_vocab.items():
    #print(n_gram, c)
    m_gram = n_gram[:-1]
    m_count = m_vocab.get(m_gram, 0)
    probabilities[n_gram] = -math.log((n_count + k) / (m_count + k * vocab_size))

probabilities

{('<s>', '<s>', 'machine'): 4.61181472870676,
 ('<s>', 'machine', 'learning'): 4.273187854639731,
 ('machine', 'learning', 'is'): 4.64971856224916,
 ('learning', 'is', 'a'): 4.817974759507122,
 ('is', 'a', 'field'): 6.082218910376446,
 ('a', 'field', 'of'): 5.648974238161206,
 ('field', 'of', 'study'): 5.658320100579444,
 ('of', 'study', 'in'): 6.05443934626937,
 ('study', 'in', 'artificial'): 6.053264948013429,
 ('in', 'artificial', 'intelligence'): 6.053264948013429,
 ('artificial', 'intelligence', 'concerned'): 6.060290738037835,
 ('intelligence', 'concerned', 'with'): 6.053264948013429,
 ('concerned', 'with', 'the'): 6.05443934626937,
 ('with', 'the', '<UNK>'): 4.9640094527562,
 ('the', '<UNK>', 'and'): 5.062595033026967,
 ('<UNK>', 'and', 'study'): 6.150602768446279,
 ('and', 'study', 'of'): 6.053264948013429,
 ('study', 'of', 'statistical'): 6.053264948013429,
 ('of', 'statistical', 'algorithms'): 6.055612366931734,
 ('statistical', 'algorithms', 'that'): 6.053264948013429,
 ('al

In [ ]:
def _best_candidate(model, prev, i, without=[]):
    """Choose the most likely next token given the previous (n-1) tokens.

    If selecting the first word of the sentence (after the SOS tokens),
    the i'th best candidate will be selected, to create variety.
    If no candidates are found, the EOS token is returned with probability 1.

    Args:
        prev (tuple of str): the previous n-1 tokens of the sentence.
        i (int): which candidate to select if not the most probable one.
        without (list of str): tokens to exclude from the candidates list.
    Returns:
        A tuple with the next most probable token and its corresponding probability.

    """
    blacklist  = ["<UNK>"] + without
    candidates = ((ngram[-1],prob) for ngram,prob in model.items() if ngram[:-1]==prev)
    #print(list(candidates))
    candidates = filter(lambda candidate: candidate[0] not in blacklist, candidates)
    candidates = sorted(candidates, key=lambda candidate: candidate[1], reverse=True)
    if len(candidates) == 0:
        return ("</s>", 1)
    else:
        return candidates[0 if prev != () and prev[-1] != "<s>" else i]


In [ ]:
num_sents = 5
min_len = 12
max_len = 24
for i in range(num_sents):
    sent, prob = ["<s>"] * max(1, N-1), 1
    while sent[-1] != "</s>":
        prev = () if N == 1 else tuple(sent[-(N-1):])
        blacklist = sent + (["</s>"] if len(sent) < min_len else [])
        next_token, next_prob = _best_candidate(probabilities, prev, i, without=blacklist)
        sent.append(next_token)
        prob += next_prob

        if len(sent) >= max_len:
            sent.append("</s>")

    print(' '.join(sent), 1/prob)

<s> <s> within a </s> 0.06917141360601399
<s> <s> ml finds application in many fields, including natural language processing, computer vision, speech recognition, machine </s> 0.010828909927066966
<s> <s> statistics and mathematical optimization methods </s> 0.030658099038941018
<s> <s> from a theoretical neural structure formed by certain </s> 0.01969099478039291
<s> <s> interest related to pattern recognition continued into the field in cognitive </s> 0.014502222747319412
